<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/Print.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
uploaded_file = files.upload()

Saving riverL.xlsx to riverL.xlsx


In [80]:
# Read the groundwater level data
gw_data = pd.read_excel('groundwater.xlsx')
gw_data['Date'] = pd.to_datetime(gw_data['Date'], format='%d/%m/%Y %H:%M')

In [81]:
print(gw_data)

                     Date      G10
0     2015-01-30 12:18:00  110.722
1     2015-01-30 13:18:00  110.720
2     2015-01-30 14:18:00  110.718
3     2015-01-30 15:18:00  110.715
4     2015-01-30 16:18:00  110.714
...                   ...      ...
17533 2017-01-30 10:19:00  109.456
17534 2017-01-30 11:19:00  109.455
17535 2017-01-30 12:19:00  109.456
17536 2017-01-30 13:19:00  109.457
17537 2017-01-30 14:19:00  109.459

[17538 rows x 2 columns]


In [82]:
 # Read the river water level data
river_data = pd.read_excel('riverL.xlsx')
river_data['Date'] = pd.to_datetime(river_data['Date'], format='%d/%m/%Y %H:%M')


In [ ]:
print(river_data)

                     Date  River
0     2014-12-31 23:00:00    192
1     2015-01-02 08:00:00    206
2     2015-01-02 20:00:00    207
3     2015-01-04 08:00:00    211
4     2015-01-04 14:00:00    207
...                   ...    ...
23772 2017-12-31 19:00:00    260
23773 2017-12-31 20:00:00    260
23774 2017-12-31 21:00:00    260
23775 2017-12-31 22:00:00    260
23776 2017-12-31 23:00:00    260

[23777 rows x 2 columns]


In [ ]:
 # Merge the two datasets on the 'Date' column
merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

In [ ]:
print(merged_data)

                     Date      G10  River
0     2015-01-30 12:18:00  110.722    261
1     2015-01-30 13:18:00  110.720    260
2     2015-01-30 14:18:00  110.718    260
3     2015-01-30 15:18:00  110.715    260
4     2015-01-30 16:18:00  110.714    260
...                   ...      ...    ...
17533 2017-01-30 10:19:00  109.456    136
17534 2017-01-30 11:19:00  109.455    138
17535 2017-01-30 12:19:00  109.456    141
17536 2017-01-30 13:19:00  109.457    144
17537 2017-01-30 14:19:00  109.459    146

[17538 rows x 3 columns]


In [ ]:
 # Set 'Date' as the index
merged_data.set_index('Date', inplace=True)

In [83]:

print(merged_data)

                         G10  River
Date                               
2015-01-30 12:18:00  110.722    261
2015-01-30 13:18:00  110.720    260
2015-01-30 14:18:00  110.718    260
2015-01-30 15:18:00  110.715    260
2015-01-30 16:18:00  110.714    260
...                      ...    ...
2017-01-30 10:19:00  109.456    136
2017-01-30 11:19:00  109.455    138
2017-01-30 12:19:00  109.456    141
2017-01-30 13:19:00  109.457    144
2017-01-30 14:19:00  109.459    146

[17538 rows x 2 columns]


In [ ]:
# Create a new date range that starts from the first timestamp in merged_data
start_date = merged_data.index.min()
end_date = merged_data.index.max()
new_date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Reindex and resample merged_data with the new date range
resampled_data = merged_data.reindex(new_date_range).ffill().bfill()
# Resample the data to hourly frequency and forward fill missing values
#resampled_data = merged_data.resample('H').ffill()



In [ ]:
print(resampled_data)

                         G10  River
2015-01-30 12:18:00  110.722  261.0
2015-01-30 13:18:00  110.720  260.0
2015-01-30 14:18:00  110.718  260.0
2015-01-30 15:18:00  110.715  260.0
2015-01-30 16:18:00  110.714  260.0
...                      ...    ...
2017-01-30 10:18:00  110.019  199.0
2017-01-30 11:18:00  110.019  199.0
2017-01-30 12:18:00  110.019  199.0
2017-01-30 13:18:00  110.019  199.0
2017-01-30 14:18:00  110.019  199.0

[17547 rows x 2 columns]


# Preprocess DATA for LSTM model input

In [ ]:
# Handling missing values
data = resampled_data.fillna(method='ffill')  # Forward fill
print(data)

                         G10  River
2015-01-30 12:18:00  110.722  261.0
2015-01-30 13:18:00  110.720  260.0
2015-01-30 14:18:00  110.718  260.0
2015-01-30 15:18:00  110.715  260.0
2015-01-30 16:18:00  110.714  260.0
...                      ...    ...
2017-01-30 10:18:00  110.019  199.0
2017-01-30 11:18:00  110.019  199.0
2017-01-30 12:18:00  110.019  199.0
2017-01-30 13:18:00  110.019  199.0
2017-01-30 14:18:00  110.019  199.0

[17547 rows x 2 columns]


In [ ]:
 # Normalizing the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [ ]:
print(data_scaled)
print(data_scaled.shape)

[[1.         0.95238095]
 [0.99738903 0.94047619]
 [0.99477807 0.94047619]
 ...
 [0.08224543 0.21428571]
 [0.08224543 0.21428571]
 [0.08224543 0.21428571]]
(17547, 2)


In [ ]:
# Define window size
n_steps = 5

X, y = [], []
for i in range(n_steps, len(data_scaled)):
  X.append(data_scaled[i-n_steps:i, :])
  y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)



In [ ]:
print (X)
print(X.shape)

[[[1.         0.95238095]
  [0.99738903 0.94047619]
  [0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]]

 [[0.99738903 0.94047619]
  [0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]
  [0.98694517 0.94047619]]

 [[0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]
  [0.98694517 0.94047619]
  [0.9843342  0.95238095]]

 ...

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]]
(17542, 5, 2)


In [ ]:
print(y)
print(y.shape)

[0.98694517 0.9843342  0.98302872 ... 0.08224543 0.08224543 0.08224543]
(17542,)


In [84]:
# Check if there are any NaN values in X
nan_in_X = np.isnan(X).any()
nan_count_X = np.isnan(X).sum()

# Check if there are any NaN values in y
nan_in_y = np.isnan(y).any()
nan_count_y = np.isnan(y).sum()

print(f"NaN values in X? {nan_in_X}")
print(f"Total number of NaN values in X: {nan_count_X}")

print(f"NaN values in y? {nan_in_y}")
print(f"Total number of NaN values in y: {nan_count_y}")

NaN values in X? False
Total number of NaN values in X: 0
NaN values in y? False
Total number of NaN values in y: 0


In [ ]:
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(train_size)
print(test_size)
print(train_size+test_size)
print(X_train.shape, 'X-train shape')
print(X_test.shape, 'X-test shape')
print(y_train.shape, 'y-train shape')
print(X_test.shape, 'y-test shape')

12279
5263
17542
(12279, 5, 2) X-train shape
(5263, 5, 2) X-test shape
(12279,) y-train shape
(5263, 5, 2) y-test shape


In [ ]:
input_shape = X_train.shape[1], X_train.shape[2]
model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        LSTM(units=50),
        Dense(1)
    ])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=40, batch_size=64)


Epoch 1/40
192/192 [==============================] - 4s 6ms/step - loss: 0.0011
Epoch 2/40
192/192 [==============================] - 1s 6ms/step - loss: 2.2345e-06
Epoch 3/40
192/192 [==============================] - 1s 6ms/step - loss: 1.6510e-06
Epoch 4/40
192/192 [==============================] - 1s 6ms/step - loss: 1.8494e-06
Epoch 5/40
192/192 [==============================] - 1s 6ms/step - loss: 1.7657e-06
Epoch 6/40
192/192 [==============================] - 1s 8ms/step - loss: 2.3092e-06
Epoch 7/40
192/192 [==============================] - 1s 7ms/step - loss: 2.7175e-06
Epoch 8/40
192/192 [==============================] - 1s 6ms/step - loss: 2.4360e-06
Epoch 9/40
192/192 [==============================] - 1s 6ms/step - loss: 2.4805e-06
Epoch 10/40
192/192 [==============================] - 1s 6ms/step - loss: 2.7004e-06
Epoch 11/40
192/192 [==============================] - 1s 6ms/step - loss: 5.4742e-06
Epoch 12/40
192/192 [==============================] - 1s 6ms/step 

In [ ]:
model.save('my_lstm_model.h5')
trained_model = model
print(trained_model)

In [ ]:
loss = trained_model.evaluate(X_test, y_test)
print(loss)


165/165 [==============================] - 1s 2ms/step - loss: 4.5934e-08
4.5934491765819985e-08


# **------------------------------Anomaly detection in new data------------------------**

In [89]:
uploaded_file = files.upload()

Saving river_a.xlsx to river_a (2).xlsx


In [91]:
#Use consistent time data, in case of missing observations use forward fill to have consistent time data
#date_format= '%Y-%m-%d %H:%M:%S'

def merge_and_resample(file_groundwater, file_river, date_format_groundwater, date_format_river):
    # Read the groundwater level data
    gw_data = pd.read_excel(file_groundwater)
    gw_data['Date'] = pd.to_datetime(gw_data['Date'], format=date_format_groundwater)

    # Read the river water level data
    river_data = pd.read_excel(file_river)
    river_data['Date'] = pd.to_datetime(river_data['Date'], format=date_format_river)

    # Merge the two datasets on the 'Date' column
    merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

    # Set 'Date' as the index
    merged_data.set_index('Date', inplace=True)

    # Create a new date range that starts from the first timestamp in merged_data
    start_date = merged_data.index.min()
    end_date = merged_data.index.max()
    new_date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Reindex and resample merged_data with the new date range
    resampled_data = merged_data.reindex(new_date_range).ffill().bfill()

    return resampled_data


def train_model(x_train, y_train, input_shape, epochs=100, batch_size=32):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        LSTM(units=50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

def save_model(model, filename='my_lstm_model.h5'):
    model.save(filename)
    return model

def evaluate_model(model, x_test, y_test):
    # Evaluate the model's performance on the test data
    # It returns a list: [loss, mse]
    loss = model.evaluate(x_test, y_test)
    return loss

def preprocess_data_for_retraining(data, n_steps):
    """
    Preprocess the data for LSTM model to predict 'G10' using past values of 'G10' and 'River'.
    - data: pandas DataFrame containing 'G10' and 'River' columns.
    - n_steps: number of time steps to use for predicting the next time step.

    Returns:
    - X, y: numpy arrays of features and labels suitable for LSTM model.
    """
    # Select only relevant columns
    relevant_data = data[['G10', 'River']].copy()

    # Handling missing values using forward fill
    relevant_data.fillna(method='ffill', inplace=True)

    # Normalizing the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(relevant_data)


    # Creating sequences
    X, y = [], []
    for i in range(n_steps, len(data_scaled)):
        X.append(data_scaled[i-n_steps:i, :])
        y.append(data_scaled[i, 0])

    X, y = np.array(X), np.array(y)
    # Split the data into training and testing sets
    train_size = int(len(X) * 0.7)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test

def detect_anomalies_g10(data, window_size, threshold=2.0):
    """
    Detects anomalies in the 'G10' column of a time series data using a moving window method.

    - data: pandas DataFrame with a datetime index and at least a 'G10' column for groundwater levels.
    - window_size: The size of the moving window.
    - threshold: The z-score threshold to identify an anomaly.

    Returns a DataFrame with an 'Anomaly' column indicating True where anomalies were detected in 'G10'.
    """
    # Calculate the moving average and standard deviation for 'G10'
    rolling_mean = data['G10'].rolling(window=window_size).mean()
    rolling_std = data['G10'].rolling(window=window_size).std()

    # Calculate the z-score for each data point in 'G10'
    data['Z-Score'] = (data['G10'] - rolling_mean) / rolling_std

    # Identify data points where the absolute z-score exceeds the threshold
    data['Anomaly'] = abs(data['Z-Score']) > threshold

    return data


In [90]:
merged_df1 = merge_and_resample('gw_a.xlsx','river_a.xlsx', '%d/%m/%Y %H:%M' ,'%d/%m/%Y %H:%M')
print(merged_df1)

                         G10  River
2019-04-09 03:30:00  110.412  230.0
2019-04-09 04:30:00  110.409  229.0
2019-04-09 05:30:00  110.407  229.0
2019-04-09 06:30:00  110.404  227.0
2019-04-09 07:30:00  110.401  227.0
...                      ...    ...
2021-01-22 06:30:00  109.197  108.0
2021-01-22 07:30:00  109.197  108.0
2021-01-22 08:30:00  109.197  108.0
2021-01-22 09:30:00  109.197  108.0
2021-01-22 10:30:00  109.197  108.0

[15704 rows x 2 columns]


In [98]:
anomaly_df = detect_anomalies_g10(merged_df1, 3, threshold=1)
print(anomaly_df)

                         G10  River   Z-Score  Anomaly
2019-04-09 03:30:00  110.412  230.0       NaN    False
2019-04-09 04:30:00  110.409  229.0       NaN    False
2019-04-09 05:30:00  110.407  229.0 -0.927173    False
2019-04-09 06:30:00  110.404  227.0 -1.059626     True
2019-04-09 07:30:00  110.401  227.0 -1.000000    False
...                      ...    ...       ...      ...
2021-01-22 06:30:00  109.197  108.0       NaN    False
2021-01-22 07:30:00  109.197  108.0       NaN    False
2021-01-22 08:30:00  109.197  108.0       NaN    False
2021-01-22 09:30:00  109.197  108.0       NaN    False
2021-01-22 10:30:00  109.197  108.0       NaN    False

[15704 rows x 4 columns]


In [99]:
# Assuming you have already run the anomaly detection function and have the resampled_data1 DataFrame

# Check if there are any anomalies
if anomaly_df['Anomaly'].any():
    # Preprocess the data for retraining
    X_train, y_train, X_test, y_test = preprocess_data_for_retraining(anomaly_df, n_steps)

    # Define the input shape for the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Retrain the model
    trained_model = train_model(X_train, y_train, input_shape, epochs=40, batch_size=64)

    # Optionally, you can evaluate and save the retrained model
    saved_model = save_model(trained_model)
    print("Model retrained due to anomaly detection.")
    evaluate_model(saved_model, X_test, y_test)
else:
    print("No anomalies detected, no retraining required.")


Epoch 1/40
172/172 [==============================] - 6s 9ms/step - loss: 0.0033
Epoch 2/40
172/172 [==============================] - 2s 9ms/step - loss: 8.1454e-06
Epoch 3/40
172/172 [==============================] - 2s 9ms/step - loss: 5.0643e-06
Epoch 4/40
172/172 [==============================] - 2s 9ms/step - loss: 5.2811e-06
Epoch 5/40
172/172 [==============================] - 2s 9ms/step - loss: 5.3260e-06
Epoch 6/40
172/172 [==============================] - 2s 9ms/step - loss: 4.9983e-06
Epoch 7/40
172/172 [==============================] - 2s 14ms/step - loss: 5.8608e-06
Epoch 8/40
172/172 [==============================] - 2s 12ms/step - loss: 5.0526e-06
Epoch 9/40
172/172 [==============================] - 2s 9ms/step - loss: 4.9758e-06
Epoch 10/40
172/172 [==============================] - 2s 9ms/step - loss: 5.8728e-06
Epoch 11/40
172/172 [==============================] - 2s 9ms/step - loss: 5.9334e-06
Epoch 12/40
172/172 [==============================] - 2s 9ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model retrained due to anomaly detection.
148/148 [==============================] - 1s 3ms/step - loss: 8.6182e-07
